# Bloat Region for Various Decay Rates

In [3]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.colors as colors
import copy
from matplotlib.colors import LinearSegmentedColormap
import ipynb.fs.defs.Functions_Equations_Equilibrium_Simulation as baseFuncs
import ipynb.fs.defs.Functions_Behavioral_Stability as stabFuncs
import ipynb.fs.defs.Functions_Equilibria_Along_Parameters as sliceFuncs

In [4]:
def triplePlane(rd_vals, param_mesh = 10, init_mesh = 10):
    para = baseFuncs.paraReset()
    param_x_label = "gamma_c"
    param_y_label = "gamma_p"
    eps = 1e-3
    bnds = ((0+eps, 1-eps), (1-eps, 0+eps))
    para['r_d'] = rd_vals[0]
    data1 = sliceFuncs.evaluatePlane(para, param_x_label, param_y_label, bnds, param_x_mesh = param_mesh, 
                          param_y_mesh = param_mesh, init_mesh = init_mesh)
    para = baseFuncs.paraReset()
    para['r_d'] = rd_vals[1]
    data2 = sliceFuncs.evaluatePlane(para, param_x_label, param_y_label, bnds, param_x_mesh = param_mesh, 
                          param_y_mesh = param_mesh, init_mesh = init_mesh)
    para = baseFuncs.paraReset()
    para['r_d'] = rd_vals[2]
    data3 = sliceFuncs.evaluatePlane(para, param_x_label, param_y_label, bnds, param_x_mesh = param_mesh, 
                          param_y_mesh = param_mesh, init_mesh = init_mesh)
    return data1, data2, data3

In [1]:
def tripleRegion(ax, df1, df2, df3, rd_vals, all_white = False, show_title = True, show_plot = True, 
                 saveFig = False, saveName = "Receding Stability Regions.png", fontsize = 12):
    # Adjust data values for colormapping
    d1 = copy.deepcopy(df1) 
    d2 = copy.deepcopy(df2) 
    d3 = copy.deepcopy(df3) 
    # The goal here is to combine the grids into one grid where the value is
    d1['stab_grid'][(df1['stab_grid'] == 1) | (df1['stab_grid'] == 3)] = 1 #There is a stable nontrivial position, could be bloat or not.
    d1['stab_grid'][(df1['stab_grid'] == 2)] = 0 # There is only bloat
    d2['stab_grid'][(df2['stab_grid'] == 1) | (df2['stab_grid'] == 3)] = 1
    d2['stab_grid'][(df2['stab_grid'] == 2)] = 0
    d3['stab_grid'][(df3['stab_grid'] == 1) | (df3['stab_grid'] == 3)] = 1
    d3['stab_grid'][(df3['stab_grid'] == 2)] = 0
    stab_grid = d1['stab_grid'] + d2['stab_grid'] + d3['stab_grid']

    # Coloring business
    if all_white:
        clist = ['#FFFFFF', '#FFFFFF', '#FFFFFF', '#FFFFFF']
    else:
        clist = ['#C79FEF', '#EAB676', '#EAB676', '#EAB676']
    patch1 = mpatches.Patch(color=clist[-1], label='Both sustainable eq. & run-away bloat possible')
    patch2 = mpatches.Patch(color=clist[0], label=r'Only run-away bloat possible')
    handles = []
    handles.append(patch1)
    handles.append(patch2)
    cmap = mpl.colors.ListedColormap(clist)
    bounds = list(np.linspace(0, 1, 4))
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    
    # Ploting
    param_x_grid = df1['param_x_grid']
    param_y_grid = df1['param_y_grid']
    
    cp = ax.contourf(param_x_grid, param_y_grid, stab_grid, cmap = cmap, levels = len(rd_vals))
    cp2 = ax.contour(cp, levels=cp.levels[1:], colors='k')

    fmt = {}
    strs = [r"$T_d = {:0.0f}$".format(1/x) for x in rd_vals]
    for l, s in zip(cp2.levels, strs):
        fmt[l] = s
    
    ax.clabel(cp2, inline=1, fmt = fmt, fontsize=10)
    
    # Text, labels, and legends
    if show_title:
        ax.set_title(r"Plot of Receding Stability Regions for Increasing $r_d$")
    ax.set_ylabel(r"Process pruning propensity ($\gamma_p$)", fontsize = fontsize)
    ax.set_xlabel(r"Process creation propensity ($\gamma_c$)", fontsize = fontsize)
    if not all_white:
        ax.legend(handles=handles, frameon = False, loc = 'upper left', bbox_to_anchor=(0.05, -0.135), fontsize = fontsize)

    # SaveFig
    if show_plot:
        if saveFig:
            fig.savefig(saveName, dpi = 700, bbox_inches = 'tight')
        else:
            plt.show()